# Credit Risk Resampling Techniques
## Student Nate Walker

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Read the CSV and Perform Basic Data Cleaning

In [3]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [4]:
# Load the data
file_path = Path('c:/FinTech/LoanStats_2019Q1.csv/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()



,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [5]:
# Shape of the Dataframe
# There are 68k rows of data by 86 columns
df.shape

(68817, 86)

In [6]:
# Define features set - Uncoded values
X = df.copy()
X.drop("loan_status", axis=1, inplace=True)
X.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,n,27.24,0.0,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,n,20.23,0.0,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,n,24.26,0.0,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,n,31.44,0.0,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,n,18.76,0.0,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [7]:
df.count().sample(11) # Count of the number of items per variable
df.count().tail(11)

num_tl_op_past_12m            68817
pct_tl_nvr_dlq                68817
percent_bc_gt_75              68817
pub_rec_bankruptcies          68817
tax_liens                     68817
tot_hi_cred_lim               68817
total_bal_ex_mort             68817
total_bc_limit                68817
total_il_high_credit_limit    68817
hardship_flag                 68817
debt_settlement_flag          68817
dtype: int64

In [8]:
# Id the variable types in the dataframe
df["loan_status"].dtypes
# The loan status is an object

dtype('O')

In [9]:
# Count the two categories in the loan status column
df["loan_status"].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

## Data Cleaning and Check for missing values

In [ ]:
df.isnull().sum()  # Check for missing values

In [ ]:
df[df == 0].count()   # identify the number of cells that have '0's

## Encode the Target Variable

In [12]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
df["loan_status_code"] = lb_make.fit_transform(df["loan_status"])
# create new column loan_status_code and encode the columns as 0 & 1's

df[["loan_status", "loan_status_code"]].head(11)

df[["loan_status", "loan_status_code"]].sample(10)
#df["loan_status_code"].dtypes

,loan_status,loan_status_code
59826,low_risk,1
36178,low_risk,1
52633,low_risk,1
34556,low_risk,1
11630,low_risk,1
20301,low_risk,1
68383,low_risk,1
18903,low_risk,1
4979,high_risk,0
28288,low_risk,1


## Binary Encoding of the Data Frame

In [13]:
# Create the Dummy Codes for the dataframe
# Binary encoding using Pandas (multiple columns)
loans_binary_encoded = pd.get_dummies(df, columns=["issue_d","home_ownership","initial_list_status", "verification_status","pymnt_plan","next_pymnt_d","hardship_flag","application_type","debt_settlement_flag"])
loans_binary_encoded.head()
loans_binary_encoded.sample(5)
# Note: data dictionary: 0 - not default / 1 - defaulted loan

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,pymnt_plan_n,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,hardship_flag_N,application_type_Individual,application_type_Joint App,debt_settlement_flag_N
34902,25000.0,0.1033,810.56,29120.0,low_risk,20.26,0.0,0.0,14.0,0.0,...,1,0,0,1,0,1,1,0,1,1
9833,16000.0,0.0646,490.10,130000.0,low_risk,9.65,0.0,0.0,14.0,0.0,...,1,0,0,1,0,1,1,1,0,1
16810,10000.0,0.1240,334.06,55000.0,low_risk,10.69,0.0,1.0,11.0,0.0,...,0,0,1,1,1,0,1,1,0,1
54504,6975.0,0.0819,219.19,44200.0,low_risk,36.49,0.0,1.0,8.0,0.0,...,1,0,0,1,0,1,1,1,0,1
20790,10000.0,0.1298,336.85,55000.0,low_risk,14.64,0.0,3.0,23.0,0.0,...,1,0,0,1,1,0,1,1,0,1


In [14]:
# Confrim if the target column is still in the dataframe
loans_binary_encoded["loan_status"].sample(5)

68676    low_risk
35663    low_risk
20404    low_risk
31814    low_risk
49371    low_risk
Name: loan_status, dtype: object

In [ ]:
# Confrim if the target column is still in the dataframe
loans_binary_encoded["next_pymnt_d"].sample(5)
#loans_binary_encoded.head()

In [16]:
# Drop the loan status column from the loans_binary_encoded df
loans_encoded = loans_binary_encoded.drop(['loan_status'],axis=1)
loans_encoded.head()
loans_encoded.tail()
#loans_encoded.sample()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,pymnt_plan_n,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,hardship_flag_N,application_type_Individual,application_type_Joint App,debt_settlement_flag_N
68812,10000.0,0.1502,346.76,26000.0,9.60,0.0,0.0,9.0,0.0,2684.0,...,0,1,0,1,0,1,1,1,0,1
68813,12000.0,0.2727,368.37,63000.0,29.07,0.0,0.0,8.0,0.0,13314.0,...,1,0,0,1,0,1,1,1,0,1
68814,5000.0,0.1992,185.62,52000.0,14.86,0.0,0.0,5.0,1.0,3715.0,...,0,1,0,1,0,1,1,1,0,1
68815,40000.0,0.0646,1225.24,520000.0,9.96,0.0,1.0,21.0,0.0,59529.0,...,0,0,1,1,0,1,1,1,0,1
68816,16000.0,0.1131,350.36,72000.0,7.02,2.0,0.0,12.0,1.0,11882.0,...,0,0,1,1,0,1,1,1,0,1


## More Data Cleaning to Get the Scaling Code to Run without Errors
### This section of code looks for variables that are not encoded
### Once I found the uncoded variables, I re-coded that varible in the encoded dataframe

In [ ]:
 # identify the number of cells that have 'w's
loans_encoded[loans_encoded == 'w'].count()  

In [ ]:
 # identify the number of cells that have 'May-2019's
loans_encoded[loans_encoded == 'May-2019'].count()  

In [ ]:
#Individual
 # identify the number of cells that have 'Individual'
loans_encoded[loans_encoded == 'Individual'].count() 

In [ ]:
## count the number of month categories
df["issue_d"].value_counts()

In [ ]:
# Id the variable types in the dataframe, looking for Object variables in the dataframe
df["issue_d"].dtypes

## Set the Target Vector

In [56]:
# Define target vector
#y = df["loan_status_code"].values.reshape(-1, 1)
#y[:5]
#y = df["loan_status"]   # Uncoded Target
y=df["loan_status_code"] # Coded Target
#y[:5]

# The targt vector here is incomplete, it does not use the values and reshape methods here. 
# See Code block #85 for correct target vector

In [18]:
# Define target vector
y2 = df["loan_status"].values.reshape(-1, 1)
y2[:5]

# y2 is the uncoded target.  I did use this target 

array([['low_risk'],
       ['low_risk'],
       ['low_risk'],
       ['low_risk'],
       ['low_risk']], dtype=object)

# Split the Data into Training and Testing

In [85]:
# Create our features
# X = # YOUR CODE HERE
X = loans_encoded

# Create our target
# y = # YOUR CODE HERE
# y coded values: 0 = low risk and 1 = high risk
y = df["loan_status_code"].values.reshape(-1, 1)
y[:5]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(loans_encoded, y, random_state=1)

In [86]:
X_train.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,pymnt_plan_n,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,hardship_flag_N,application_type_Individual,application_type_Joint App,debt_settlement_flag_N
count,51612.000000,51612.000000,51612.000000,5.161200e+04,51612.000000,51612.000000,51612.000000,51612.000000,51612.000000,51612.000000,...,51612.000000,51612.000000,51612.000000,51612.0,51612.000000,51612.000000,51612.0,51612.000000,51612.000000,51612.0
mean,16711.575312,0.127699,481.504593,8.847633e+04,21.740295,0.218515,0.496241,12.582558,0.125804,17638.211734,...,0.478939,0.372801,0.148260,1.0,0.384465,0.615535,1.0,0.859839,0.140161,1.0
std,10291.735620,0.048201,288.491457,1.233078e+05,18.901909,0.715931,0.757929,6.016046,0.335870,21927.460755,...,0.499561,0.483554,0.355361,0.0,0.486473,0.486473,0.0,0.347158,0.347158,0.0
min,1000.000000,0.060000,30.890000,1.200000e+02,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,1.0,0.000000,0.000000,1.0
25%,9193.750000,0.088100,265.730000,5.015500e+04,13.870000,0.000000,0.000000,8.000000,0.000000,6296.000000,...,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,1.0,1.000000,0.000000,1.0
50%,15000.000000,0.118000,404.790000,7.300000e+04,19.730000,0.000000,0.000000,11.000000,0.000000,12078.500000,...,0.000000,0.000000,0.000000,1.0,0.000000,1.000000,1.0,1.000000,0.000000,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.670000,0.000000,1.000000,16.000000,0.000000,21707.500000,...,1.000000,1.000000,0.000000,1.0,1.000000,1.000000,1.0,1.000000,0.000000,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,563245.000000,...,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.0,1.000000,1.000000,1.0


In [88]:
#y = df["loan_status_code"].values.reshape(-1, 1)
y[:5]

array([[1],
       [1],
       [1],
       [1],
       [1]])

In [31]:
# Check the balance of our target values
y.value_counts()  # Untarget - y

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

## Create the Standard Scaler

In [81]:
# Create X_train, X_test, y_train, y_test
# YOUR CODE HERE
# Create the StandardScaler instance
scaler = StandardScaler()

In [82]:
# Fitting the scaler
scaler.fit(loans_encoded)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [83]:
# Fit the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)
#y_scaler = scaler.fit(y_train)

In [89]:
y_scaler = scaler.fit(y_train)

In [90]:
# Scale the training data
X_train_scaled = X_scaler.transform(X_train)

X_test_scaled = X_scaler.transform(X_test)
y_test_scaled = y_scaler.transform(y_test)

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [91]:
# Resample the training data with the RandomOversampler
# YOUR CODE HERE
# implement random oversampling
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({1: 51366, 0: 51366})

In [92]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [94]:
# Display the confusion matrix
# YOUR CODE HERE
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   69,    32],
       [ 6435, 10669]], dtype=int64)

In [95]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred)

0.6534702669284702

In [96]:
# Print the imbalanced classification report
# YOUR CODE HERE
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.68      0.62      0.02      0.65      0.43       101
          1       1.00      0.62      0.68      0.77      0.65      0.42     17104

avg / total       0.99      0.62      0.68      0.76      0.65      0.42     17205



### SMOTE Oversampling

In [97]:
# Resample the training data with SMOTE
# YOUR CODE HERE
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train, y_train
)
from collections import Counter

Counter(y_resampled)

Counter({1: 51366, 0: 51366})

In [98]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [99]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.654920625364688

In [100]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, y_pred)

array([[   65,    36],
       [ 5708, 11396]], dtype=int64)

In [101]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.64      0.67      0.02      0.65      0.43       101
          1       1.00      0.67      0.64      0.80      0.65      0.43     17104

avg / total       0.99      0.67      0.64      0.79      0.65      0.43     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [102]:
# Resample the data using the ClusterCentroids resampler
# YOUR CODE HERE
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 246, 1: 246})

In [103]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
model = LogisticRegression(solver='lbfgs', random_state=1, max_iter=2000)
model.fit(X_resampled, y_resampled)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=2000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [104]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
from imblearn.metrics import classification_report_imbalanced

y_pred = model.predict(X_test)
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.79      0.50      0.02      0.63      0.41       101
          1       1.00      0.50      0.79      0.66      0.63      0.38     17104

avg / total       0.99      0.50      0.79      0.66      0.63      0.38     17205



In [105]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, y_pred)

array([[  80,   21],
       [8592, 8512]], dtype=int64)

In [106]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.79      0.50      0.02      0.63      0.41       101
          1       1.00      0.50      0.79      0.66      0.63      0.38     17104

avg / total       0.99      0.50      0.79      0.66      0.63      0.38     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [107]:
# Resample the training data with SMOTEENN
# YOUR CODE HERE
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 51359, 1: 46660})

In [108]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [109]:
# Display the confusion matrix
# YOUR CODE HERE
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[  75,   26],
       [7771, 9333]], dtype=int64)

In [110]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred)

0.6441180454574924

In [111]:
# Print the imbalanced classification report
# YOUR CODE HERE
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.74      0.55      0.02      0.64      0.41       101
          1       1.00      0.55      0.74      0.71      0.64      0.40     17104

avg / total       0.99      0.55      0.74      0.70      0.64      0.40     17205



## Model Interpretaions
## Question #1: Which Model Produced the Best Balanced Accuracy Score (BAS)?

In [ ]:
# Question #1: Which Model Produced the Best Balanced Accuracy Score (BAS)?
"""
The Random Over Samping Model: BAS = 0.653  
The SMOTE Over Sampling Model: BAS = 0.655 
The Combined Model:            BAS = 0.644 
The Undersampled Model:        BAS = n/a.  

The Best BAS is associated with the SMOTE Model. This model has the highest BAS.

Intepretation: The SMOTE Model BAS of 65.5%, means that only 65.5% of the predicted values were
correct. 
"""

## Question #2: Which Model has the Best Recall Score (REC)?

In [117]:
"""
The Random Over Samping Model: REC = 0.62  
The SMOTE Over Sampling Model: REC = 0.67 
The Combined Model:            REC = 0.55 
The Undersampled Model:        REC = 0.50  

The Best REC is associated with the SMOTE Model. This model has the highest REC.
Intepretation: The SMOTE model again, is better at predicting the highest proportion 
of actual postive results of predicting high risk loans. 
"""

'\nThe Random Over Samping Model: REC = 0.62  \nThe SMOTE Over Sampling Model: REC = 0.67 \nThe Combined Model:            REC = 0.55 \nThe Undersampled Model:        REC = 0.50  \n\nThe Best REC is associated with the SMOTE Model. This model has the highest REC\n'

## Question #3: Which Model has the Best Geometric Mean Score (GEO)?

In [118]:
"""
The Random Over Samping Model: GEO = 0.65  
The SMOTE Over Sampling Model: GEO = 0.65 
The Combined Model:            GEO = 0.64 
The Undersampled Model:        GEO = 0.63  

The Best  isGEO associated with the Random Oversampling Model and the SMOTE Model. 
These two models haves the highest GEO Scores.
Intepretation: All of the GEO Scores are centered around 0.64 .  This represents the 
central tendency score of all of the values in the set of numbers. 
"""

'\nThe Random Over Samping Model: GEO = 0.65  \nThe SMOTE Over Sampling Model: GEO = 0.65 \nThe Combined Model:            GEO = 0.64 \nThe Undersampled Model:        GEO = 0.63  \n\nThe Best  isGEO associated with the Random Oversampling Model and the SMOTE Model. \nThese two models haves the highest GEO Scores.\n'

## Final Comments:

In [119]:
"""
The analysis has showed that for the models that were used,
there is no difference in the results and that the models
would need to be improved by using additional data or
look for correlated values to reduce the high dimensionality
to the categories that best describe the data structure.
"""

'\nThe analysis has showed that for the models that were used,\nthere is no difference in the results and that the models\nwould need to be improved by using additional data or\nlook for correlated values to reduce the high dimensionality\nto the categories that best describe the data structure.\n'